<!-- <p style="text-align:center;"> -->
#Highway to Hell (Or Heaven)

                

# Introduction (20%)


Two plants share a common logistics network that includes ports, carriers, warehouses, shops, and products. Both plants manufacture the same type of product, but their production methods differ, resulting in varying production costs. However, because the market is competitive, both plants must sell their products at the same price. Since one plant uses a cheaper production method, it carries a higher risk of defects.

Additionally, each warehouse connected to a shop must contain at least 140% of the quantity of good (non-defective) products available in that shop. Also, the earlier plants introduce their product to the market, the more customers have the opportunity to see it, and it is more likely to be purchased by them. If a plant uses a carrier that has faster delivery, it can get its products to the market soon, which in turn comes with a cost. Each warehouse also has a storage cost, but the warehouse does not accept any defective products for storage because of safety regulations. The main objective is to determine the optimal logistics path for each plant that maximizes overall profit.


- Talk about the use case of these kind of optimisations




This is a depiced graph of the scenario
![alt text](https://github.com/ashvin-a/Final-Project-524/blob/main/docs/Logistic-graph.svg?raw=1)

In [22]:
# Data Analysis
sheet_names = ["OrderList",
               "FreightRates",
               "WhCosts",
               "WhCapacities",
               "ProductsPerPlant",
               "VmiCustomers",
               "PlantPorts"]

import pandas as pd
df = pd.read_excel("file.xlsx", sheet_name=sheet_names)
df

{'OrderList':           Order ID Order Date Origin Port Carrier  TPT Service Level  \
 0     1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
 1     1.447158e+09 2013-05-26      PORT09   V44_3    1           CRF   
 2     1.447139e+09 2013-05-26      PORT09   V44_3    1           CRF   
 3     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 4     1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 ...            ...        ...         ...     ...  ...           ...   
 9210  1.447305e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9211  1.447319e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9212  1.447322e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9213  1.447145e+09 2013-05-26      PORT04  V444_1    1           DTD   
 9214  1.447328e+09 2013-05-26      PORT04  V444_1    1           DTD   
 
       Ship ahead day count  Ship Late Day count           Customer  \
 0                        3           

As we can observe, the excel sheet consists of 7 sheets. Here is my description on what each sheets tells us about:

- OrderList: Contains unique identifier as order id.

In [23]:
for index, dataframe in df.items():
  print(f"\n\n\nTable name: {index}\n Dataframe Summary : {dataframe.columns}\n")
  for column in dataframe.columns:
    if column not in ["Order ID", "Product ID", "minimum cost", "max_wgh_qty", "minm_wgh_qty"]:
      print(f"Column name : {column}\n ")
      # Unique Entries :{dataframe[column].unique()}




Table name: OrderList
 Dataframe Summary : Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')

Column name : Order Date
 
Column name : Origin Port
 
Column name : Carrier
 
Column name : TPT
 
Column name : Service Level
 
Column name : Ship ahead day count
 
Column name : Ship Late Day count
 
Column name : Customer
 
Column name : Plant Code
 
Column name : Destination Port
 
Column name : Unit quantity
 
Column name : Weight
 



Table name: FreightRates
 Dataframe Summary : Index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'minm_wgh_qty',
       'max_wgh_qty', 'svc_cd', 'minimum cost', 'rate', 'mode_dsc',
       'tpt_day_cnt', 'Carrier type'],
      dtype='object')

Column name : Carrier
 
Column name : orig_port_cd
 
Column name : dest_port_cd
 
Column name : svc_cd
 
Co

In [ ]:
for index, dataframe in df.items():
  if index == "OrderList":
    for column in dataframe.columns:
      if column not in ["Order ID", "Product ID", "minimum cost", "max_wgh_qty", "minm_wgh_qty"]:
        print(f"Column name : {column}")

# Approach (15%)
- How the heck are we gonna solve this.
- Be intuitive and cool


# The Freaking Solution (30%)


In [ ]:
# Data Preprocessing (10%)

In [ ]:
!pip install gamspy
!gamspy install license 2a571695-fbd3-4aa6-9587-b3d6083d1be8

In [ ]:
# Do our implementations here

In [ ]:
# Sensitivity Analysis, Visualization (10%)

# Conclusion of the Project

# Extensions of the project
